Calculate and save the hitting time matrices for brain networks for all the subjects:

This code runs on fMRI BOLD measurments from UCLA study. 
- Input the task setting and the atlas to be used for graph representation
- The data has already been de-faced and in MNI spapce
Steps:
1. Get rid of the motion paramaters for every voxel
2. Map the data to regions specified by the atlas (reducing the dimensionality of fMRI to the number of ROIs)
3. Calculate the similarity matrix to be partial correlations
4. Threshold the similarity matrix to remove weaker connections
5. Build the graph by calculating the adjacency matrix
6. Find the mean first passage time between every 2 nodes in the graph and save it as the hitting time matrix

In [1]:
!jupyter nbconvert sharedPaul.ipynb --to html

Error executing Jupyter command 'nbconvert': [Errno 2] No such file or directory


In [ ]:
### inputs ###
task = "rest" # rest or bart
atLas = "mmp" # msdl or haox or mmp

In [ ]:
import os
%matplotlib inline
import nilearn
import numpy as np
import scipy
import math
from nilearn import datasets
from nilearn.input_data import NiftiMapsMasker
from nilearn.input_data import NiftiLabelsMasker
from nilearn.connectome import ConnectivityMeasure
import nibabel as nib
import openpyxl
import matplotlib.pyplot as plt
import csv
import pandas as pd
from numpy import genfromtxt
correlation_measure = ConnectivityMeasure(kind='partial correlation')

In [ ]:
## reading the subject's ID's and groups
wb = openpyxl.load_workbook('Subjects1.xlsx')
wb.get_sheet_names()
sheet = wb.get_sheet_by_name('Sheet1')
N = sheet.max_row

In [ ]:
# directory to raw fMRI measurements
Data_DIR = "F://UCLA//func/"
os.chdir(Data_DIR)
os.getcwd()

In [ ]:
# motCor function to correct for motion
# inputs: subject ID and task
# output: data confounds containing the motion regressors
def motCor(sub, task):
    # recovering file name from subject ID and task
    Data_Name = "sub-" + sub + "_task-" + task + "_bold_space-MNI152NLin2009cAsym_preproc"    
    # set the name of the desired data set here:
    fmri_filenames = Data_Name + "_filt.nii.gz"
    # the file containing motion parameters
    tsv_file = "sub-" + sub+ "_task-" + task + "_bold_confounds.tsv" 
    df = pd.read_table(tsv_file, sep='\t', header=0) 
    [dim1, dim2] = np.shape(df)
    # identify which motion parameters to use
    conf_clmns = [row for row in df if row == "WhiteMatter" or row == "X" or row == "Y" or row == "Z"]
    data_confs = np.zeros((dim1, 4))
    cntr = 0
    for item in conf_clmns:
        data_confs[:, cntr] = np.array(df[item])
        cntr = cntr + 1
    # removing the first 10 seconds of fMRI (the sharp transition part of fMRI)
    data_confs = data_confs[5:,:]
    return data_confs

In [ ]:
# dimensionality reduction function maps the fMRI data on MMP-HCP atlas with 180 regions
# inputs: subject ID and task and motion confounds
# output: the time series corresponding to 180 regions of MMP-HCP (Glasser-2016) atlas
def dimRed(sub, task, confounds):
    # recovering file name from subject ID and task
    Data_Name = "sub-" + sub + "_task-" + task + "_bold_space-MNI152NLin2009cAsym_preproc"
    # file containing the 180 ROIs map
    atlas_filename = "MMP1_rois.nii"
    
    masker = NiftiLabelsMasker(labels_img=atlas_filename, standardize=True,
                               memory='nilearn_cache', verbose=5) 
    
    # set the name of the desired data set here:
    fmri_filenames = Data_Name + "_filt.nii.gz"
    time_series = masker.fit_transform(filenames, confounds = data_confs)
    return time_series

In [ ]:
# finding the adjacency matrix: threshold the fMRI time series and map it to a graph with 180 nodes and 
# inputs: correlation (partial correlation) matrix and the desired percentile to be used to find the threshold
# output: adjacency matrix
def adjMatrix(C_matrix, percentile):
    # find the magnitude of the correlations
    A_matrix = abs(C_matrix)
    A_vec = np.reshape(A_matrix, (1, 180*180))
    
    # remove the self loops
    np.fill_diagonal(A_matrix, 0)
    
    # calculate the threshold from desired percentile
    thrshld = np.percentile(A_vec, 30)
    L = np.size(A_matrix,axis = 0)
    
    # threshold the adjacency matrix
    A_matrix = np.array(A_matrix)
    low_values_flags = A_matrix < thrshld
    A_matrix[low_values_flags] = 0
    return A_matrix

In [ ]:
# calculating mean first passage time of the random walk (also known as hitting time) from the normalized Laplacian
# input: adjacency matrix
# output: hitting time matrix
def meanfstpsg(A_matrix):
    L = np.size(A_matrix,axis = 0)
    
    D_matrix = np.zeros((L,L)) 
    D_inv = np.zeros((L,L)) 
    D_sqrt = np.zeros((L,L)) 
    D_sqrt_inv = np.zeros((L,L)) 
    for i in range(L):
        D_matrix[i,i] = np.sum(A_matrix[i]) # degree matrix
        D_inv[i,i] = 1./D_matrix[i,i] # inverse of degree matrix
        D_sqrt[i,i] = np.sqrt(D_matrix[i,i]) # second root of degrees
        D_sqrt_inv[i,i] = 1./D_sqrt[i,i] # inverse of the second roots of degrees 
    
    # the probability transitioning matrix: P = A/D
    p_matrix = np.dot(D_inv, A_matrix)
    
    # normalized graph Laplacian
    eye_matrix = np.eye(L,L)
    eye_P = eye_matrix - p_matrix

    G_Lap = np.dot(D_sqrt,eye_P)
    G_Lap_n = np.dot(G_Lap, D_sqrt_inv)
    
    # eigen decomposition of the graph Laplacian
    [eig_val, eig_vec] = np.linalg.eigh(G_Lap_n)
    
    # calculate the eigen matrix for hitting time calculations - to reduce the computation time
    eig_matrix = [] # the columns of the matrix are the eigenvectors divided by the corresponding eigenvalue
    for k in range(1, L): # the first eigenvalue is zero
        eigmatrix.append(np.divide(eig_vec, math.sqrt(eig_val[k])), 1)
    
    # calculating the hitting times
    H = np.zeros((L,L))
    d = np.sum(D_matrix)
    for i in range(L):
        deg_i = D_matrix[i,i]
        t_1 = (d/d_i)*(eigmatrix[i,:]*np.transpose(eigmatrix[i,:]))
        for j in range(L):
            deg_j = D_matrix[j,j]
            t_2 = (d/(d_i*d_j))*(eigmatrix[j,:]*np.transpose(eigmatrix[i,:]))
            H[i,j] = t_1 - t_2
    return H

In [ ]:
## calculating and saving hitting-time matrix for every subject
for i in range(1,N):
    # subject ID:
    ID = sheet.cell(row=i+1, column=1).value
    sub = str(ID) 
    # motion regressors:
    data_confs = motCor(sub, task)
    # time series for 180 ROIs:
    time_series = dimRed(sub, task, data_confs)
    # similarity matrix:
    correlation_matrix = correlation_measure.fit_transform([time_series])[0]  
    # adjacency matrix
    A_matrix = adjMatrix(correlation_matrix, percentile = 30)
    # hitting time matrix:
    H = meanfstpsg(A_matrix)
    
    np.savetxt("hit_time_task_" + atLas + "_sub_data_7p_filt_" + Data_Name, H, delimiter=",")